In [1]:
authorList = [551, 2703, 2971, 8303, 7679]
doc_id = 1210

In [ ]:
parameters = {
    'dropout': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'samples': [50, 100, 150, 200, 250, 300],
    'dimensions': [50, 100, 200], # 300],
    'iterations': [20, 40, 60, 80, 100],
}
results = {
    'dropout': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    'samples': [0, 0, 0, 0, 0, 0],
    'dimensions': [0, 0, 0, 0],
    'iterations': [0, 0, 0, 0, 0],
}
for idxp, paralist in parameters.iteritems():
    if idxp == 'dropout':
        for idxl, val in enumerate(paralist):
            import StyloNeural as Stylo
            (labels_index, predYList, predY, history) = Stylo.getResults(authorList=authorList, 
                                                                         doc_id=2178, 
                                                                         level = 'word', 
                                                                         glove = '../../glove/',
                                                                         nb_epoch = 40,
                                                                         samples = 300, 
                                                                         dimensions = 100,
                                                                         dropout = val
                                                                        )
            results[idxp].insert(idxl, predY)
        
    
    if idxp == 'samples':
        for idxl, val in enumerate(paralist):
            import StyloNeural as Stylo
            (labels_index, predYList, predY, history) = Stylo.getResults(authorList=authorList, 
                                                                         doc_id=2178, 
                                                                         level = 'word', 
                                                                         glove = '../../glove/',
                                                                         nb_epoch = 40,
                                                                         samples = val, 
                                                                         dimensions = 100,
                                                                         dropout = 0.4
                                                                        )
            results[idxp].insert(idxl, predY)
        
    
    if idxp == 'dimensions':
        for idxl, val in enumerate(paralist):
            import StyloNeural as Stylo
            (labels_index, predYList, predY, history) = Stylo.getResults(authorList=authorList, 
                                                                         doc_id=2178, 
                                                                         level = 'word', 
                                                                         glove = '../../glove/',
                                                                         nb_epoch = 40,
                                                                         samples = 300, 
                                                                         dimensions = val,
                                                                         dropout = 0.4
                                                                        )
            results[idxp].insert(idxl, predY)
        
    
    if idxp == 'iterations':
        for idxl, val in enumerate(paralist):
            import StyloNeural as Stylo
            (labels_index, predYList, predY, history) = Stylo.getResults(authorList=authorList, 
                                                                         doc_id=2178, 
                                                                         level = 'word', 
                                                                         glove = '../../glove/',
                                                                         nb_epoch = val,
                                                                         samples = 300, 
                                                                         dimensions = 100,
                                                                         dropout = 0.4
                                                                        )
            results[idxp].insert(idxl, predY)

Using Theano backend.


In [ ]:
print results

In [ ]:
# import matplotlib.pyplot as plt
# # summarize history for accuracy
# plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

In [ ]:
# # summarize history for loss
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()